# kCSD-python package tutorial

In this tutorial we illustrate the use of kCSD-python package for CSD reconstruction in the simplest case of a regular 2D square grid. This is a simplified version of a slice on a microelectrode array [Ness et al., 2015], or a planar silicone probe within the brain, where we assume constant conductivity in the whole space.

We start with the basic CSD estimation on a regular grid. First, we define a region of interest. Then, using predefined test functions for the current sources, we place a ground truth current source in this region. We define the distribution of electrodes. Assuming ideal electrodes, we compute the potential generated by the selected current sources as measured at the electrodes. Given these potentials and the electrode locations we estimate the current source density using kCSD (Basic features). In the second part of this tutorial, we explore noisy electrodes and test the robustness of the method (Noisy electrodes). In the final part we look at how the errors in the estimation depend on the sources and the electrode configuration by testing the effect of broken electrodes on reconstruction (Broken electrodes).

## Basic features
In this tutorial we illustrate the use of kCSD package for CSD reconstruction in the simplest case of a regular 2D square grid. The following steps, will take us through this.
1. Define a region of interest - a unit square as the test bench
2. Place one of predefined current sources in this region
3. Define electrode configuration in the the region of interest
4. Assuming ideal electrodes, obtain the potential generated at the electrodes by the introduced current sources
5. Estimate current source density from the measured potentials given the electrode locations and using the default parameters
6. Perform cross validation to improve the estimate and avoid overfitting

### Region of interest

We use the `numpy` Python library, and define a 2 dimensional grid using the `meshgrid` function. We define the region of interest between `0` and `1` (inclusive) in the `xy` plane, with a resolution of `100` points in each dimension (as indicated by `100`). The units we will assume are in $mm$.

In [ ]:
import numpy as np

csd_at = np.meshgrid(np.linspace(0, 1, 100),
                     np.linspace(0, 1, 100),
                     indexing='ij')
csd_x, csd_y = csd_at

### Predefined current sources for testing

In [ ]:
from kcsd import csd_profile as CSD


CSD_PROFILE = CSD.gauss_2d_small
true_csd = CSD_PROFILE(csd_at, seed=15) 

The `kCSD-python` library provides functions to generate test sources which can be imported from the `csd_profile` module. Here we use the `gauss_2d_small` function to generate two-dimensional Gaussian sources which are small in the scale set by the interelectrode distance. The other implemented option for two-dimensional test sources is the `gauss_2d_large` function. To generate the exact same sources in each run we must invoke these functions using the same random seed which is stored in the seed variable. For simplicity, these current sources are static and do not change with time. 
***
We visualize the current sources as a heatmap. Note that changing the `seed`, or the source type to `gauss_2d_large`, changes the type of the current sources generated. The code below displays this test source as the True CSD. For convenience we define this as a function `make_plot`.
***
The first line starting with the `%` is to ensure the plots using this tutorial are displayed correctly.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm


def make_plot(xx, yy, zz, title='True CSD', cmap=cm.bwr):
    fig = plt.figure(figsize=(7, 7))
    ax = plt.subplot(111)
    ax.set_aspect('equal')
    t_max = np.max(np.abs(zz))
    levels = np.linspace(-1 * t_max, t_max, 32)
    im = ax.contourf(xx, yy, zz, levels=levels, cmap=cmap)
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_title(title)
    ticks = np.linspace(-1 * t_max, t_max, 3, endpoint=True)
    plt.colorbar(im, orientation='horizontal', format='%.2f', ticks=ticks)
    return ax


make_plot(csd_x, csd_y, true_csd, title='True CSD', cmap=cm.bwr)




### Place electrodes

In [ ]:
ele_x, ele_y = np.meshgrid(np.linspace(0.05, 0.95, 10),
                           np.linspace(0.05, 0.95, 10))

ele_pos = np.vstack((ele_x.flatten(), ele_y.flatten())).T

We now define the virtual electrodes within the region of interest. We place them between `0.05` $mm$ and `0.95` $mm$ of the region of interest, with a resolution of `10` (as indicated by `10` in `meshgrid`) in each dimension, totalling to `100` electrodes. Notice that the electrodes do not span the entire region of interest. Although in this example the electrodes are distributed on a regular grid, this is not required by the kCSD method as it can handle arbitrary distributions of electrodes.

### Compute potential

In [ ]:
from scipy.integrate import simps

CONDUCTIVITY = 1.0 # S / m
H = 50. # distance between the electrode plane and the bottom of the slice


def integrate_2d(csd_at, true_csd, ele_pos, h, csd_lins):
    csd_x, csd_y = csd_at
    xlin, ylin = csd_lins

    Ny = ylin.shape[0]
    r = np.sqrt((ele_pos[0] - csd_x)**2 + (ele_pos[1] - csd_y)**2)
    r[r < 0.0000001] = 0.0000001
    weighted_csd = np.arcsinh(h / r) * true_csd
    
    integral_1D = [simps(col, ylin) for col in weighted_csd.T]
    return simps(integral_1D, xlin)


def forward_method(ele_pos, csd_at, true_csd):
    lin_x = csd_x[:, 0]
    lin_y = csd_y[0, :]
    
    integrals = np.array([integrate_2d(csd_at, true_csd,
                                       pos, H,
                                       [lin_x, lin_y])
                          for pos in ele_pos])
    return integrals / (2 * np.pi * CONDUCTIVITY)


pots = forward_method(ele_pos, csd_at, true_csd)

At the given electrode positions we need to evaluate the potential generated by the current sources that were placed in step 2, this is given by the function `forward_method`. We make some assumptions regarding the location of the sources with respect to the electrode positions, given by the global variable `H`. We also assume infinite and homogeneous medium whose conductivity is given by a global variable `CONDUCTIVITY` and equals `1` $S/m$. We utilize the `scipy` library's `simps` function to integrate the current sources in the two dimensions to obtain the potentials, `pots`, at the electrode positions. We also assume that the electrodes are ideal, point-size and noise free.
***
To visualize the potential, we interpolate the hundred values computed at the electrodes positions with `scipy` libraries `interpolate.griddata` function. Note that the kCSD estimation uses only the potential recorded at the electrode positions. To distinguish between the potentials and CSD plots we use different colormaps. The electrodes are marked with dots in this plot.

In [ ]:
from scipy.interpolate import griddata


def grid(x, y, value):
    x = x.flatten()
    y = y.flatten()
    value = value.flatten()
    x_i, y_i = np.meshgrid(np.linspace(x.min(), x.max(), 100),
                           np.linspace(y.min(), y.max(), 100),
                          indexing='ij')                                                                                                                                                        
    value_i = griddata((x, y), value, (x_i, y_i),
                       method='linear')
    return x_i, y_i, value_i


pot_X, pot_Y, pot_Z = grid(ele_pos[:, 0],
                           ele_pos[:, 1],
                           pots)

ax = make_plot(pot_X, pot_Y, pot_Z,
               title='Interpolated potentials',
               cmap=cm.PRGn)
ax.scatter(ele_pos[:, 0], ele_pos[:, 1], 10, c='k')

In [ ]:
from kcsd import KCSD2D


def do_kcsd(ele_pos, pots):
    pots = pots.reshape((len(ele_pos), 1)) # one time point
    return KCSD2D(ele_pos, pots,
                  h=H,
                  sigma=CONDUCTIVITY,
                  xmin=0.0,
                  xmax=1.0,
                  ymin=0.0,
                  ymax=1.0,
                  n_src_init=1000,
                  src_type='gauss',
                  R_init=1.)


k = do_kcsd(ele_pos, pots)
est_csd = k.values('CSD')

Here we illustrate the most basic estimation of CSD with the `kcsd`
library. Since our example is two dimensional the relevant method is KCSD2D. For convenience we encapsulate the actual method call with parameters being set inside a function `do_kcsd` where we perform the following steps: 
* We first set h and sigma parameters of the forward model. 
* Then we restrict the potentials to the first time point of the recording. For typical experimental data the shape of this matrix would be $N_{ele} \times N_{time}$, where $N_{ele}$ is the number of electrodes and $N_{time}$ is the total number of recorded time points. 
* Next, we call the `KCSD2D` class with the relevant parameters. The only required parameters are the electrode positions, `ele_pos`, and the potentials they see, `pots`. 
* We can also provide here the parameters for the forward model, `h` and `sigma`. 
* We define a rectangular region of estimation by setting the values `xmin`, `xmax` and `ymin`, `ymax`. The number of basis functions, `n_src_init` is set to `1000`, basis functions are of the type `gauss`, and the width of the Gaussian basis source `R_init` is set to be `1`. 
* Finally, estimated `CSD` is stored as `est_csd`.
***
Observe that the estimation is not very faithful when compared to the True CSD obtained before. This is caused by the ground truth varying significantly in the scale of a single inter-electrode distance. In the next step we will use cross-validation to select better reconstruction parameters. 

In [ ]:
make_plot(k.estm_x, k.estm_y,
          est_csd[:, :, 0], # First time point
          title='Estimated CSD without CV', cmap=cm.bwr)

### Cross validation

Leave-one-out cross-validation is performed with a single line command. In this procedure we scan a range of `R` values which set the size of the Gaussian basis functions and the regularization parameter $\lambda$ values. At the end of this step we obtain the optimal parameters that would correct for overfitting. The function outputs the progress of the cross-validation step and displays the optimal candidates in the last line. Alternatively, one could use the L-curve method to find these optimal parameters. 
***
We find that the estimation of the current sources obtained after cross-validation resembles the True CSD better.

In [ ]:
k.cross_validate(Rs=np.linspace(0.15, 0.2, 6),
                 lambdas=np.logspace(-6, -5, 5))

In [ ]:
est_csd = k.values('CSD')

make_plot(k.estm_x, k.estm_y,
          est_csd[:, :, 0], # First time point
          title='Estimated CSD with CV',
          cmap=cm.bwr)

## Noisy electrodes

Until now we assumed noise-free data, however, experimental data are always noisy. In this section we investigate how noise affects the kCSD estimation. 
* We first show how to compute reliability map which is a measure of reconstruction quality. 
* Then we discuss reproducible generation of noisy data with varying noise amplitude. 
* Finally, we study the error in the reconstruction as a function of changing noise level.

### Reconstruction quality measure

To assess the estimation quality we measure the point-wise difference between the true sources and the sources reconstructed with the kcsd. We define a function `point_errors` which takes the `true_csd` and the `estimated_csd` as the inputs, normalizes them individually, and computes the Frobenius norm of their difference.
***
We visualize this difference as before, except we use greyscale colormap to display the intensity of the reconstruction error. For convenience we define the plotting in a function called `make_error_plot`.

In [ ]:
def point_errors(true_csd, est_csd):
    return np.abs(est_csd.reshape(true_csd.shape) - true_csd) / np.abs(true_csd).max()


def make_error_plot(xx, yy, error, title='Error CSD'):
    fig = plt.figure(figsize=(7, 7))
    ax = plt.subplot(111)
    ax.set_aspect('equal')
    t_max = np.max(np.abs(error))
    levels = np.linspace(0, t_max, 32)
    im = ax.contourf(xx, yy, error,
                     levels=levels,
                     cmap=cm.Greys)
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_title(title)
    ticks = np.linspace(0, t_max, 3,
                        endpoint=True)
    plt.colorbar(im,
                 orientation='horizontal',
                 format='%.2f',
                 ticks=ticks)
    return ax

In [ ]:
error_ideal = point_errors(true_csd, est_csd)
make_error_plot(k.estm_x, k.estm_y, error_ideal,
                title='Error CSD, no noise')

### Noise definition

To study resilience of the reconstruction against the noise in a controlled way we seed the random number generator in the function `add_noise`. We consider normally distributed noise with the mean and standard deviation set by reference to the recorded potentials.

In [ ]:
def add_noise(pots, noise_level=0, noise_seed=23):
    rstate = np.random.RandomState(noise_seed)
    return rstate.normal(loc=pots,
                         scale=np.mean(np.abs(pots)) * noise_level / 100)

### Source reconstruction from noisy data

With these tools we can study the effects of noise on the reconstruction. We now generate noise for a given noise level (percents of an average absolute value of potential), add it to the simulated potential, and estimate CSD from these noisy potentials. We can then use the error plots to compare the reconstruction with the True CSD. Notice that the parameters giving best reconstruction obtained for noisy data in general will be different from those obtained for clean potentials to compensate for noise.
***
We can display this error with the `make_error_plot` plotting function we defined earlier. Changing the `noise_level` and the `noise_seed` affects the reconstruction, but the error depends also on the sources, so changing the True CSD type to a `gauss_2d_large` or changing `csd_seed` will lead to different results.

In [ ]:
pots_noise = add_noise(pots,
                       noise_level=5,
                       noise_seed=23)

k_noise = do_kcsd(ele_pos, pots_noise)
k_noise.cross_validate(Rs=np.linspace(0.3, 0.35, 6),
                       lambdas=np.logspace(-6, -4, 5))

In [ ]:
estm_csd_noise = k_noise.values('CSD')
error_noise = point_errors(true_csd, estm_csd_noise)

make_plot(k_noise.estm_x, k_noise.estm_y, estm_csd_noise[:, :, 0], 
          title='CSD estimated from noisy measurements')
make_error_plot(k_noise.estm_x, k_noise.estm_y, error_noise, 
                title='Error CSD (noisy measurements)')

## Broken electrodes

It often happens that one needs to discard recordings from a subset of setup. This can happen when some electrodes are used for stimulation and cannot be used for recording, or for data managing purposes the bandwidth limitations may require a compromise between sampling rates and the number of electrodes being monitored simultaneously, or electrode may break down or get too noisy and their signals must be discarded. Here we discuss how to handle such cases and how to investigate the incurred errors in reconstruction. 
* We first show how we remove recordings from selected (broken) electrodes from considered data. 
* Then we calculate the estimation error for a given source for data from a damaged setup. 
* Finally, we compute the average error across many sources from incomplete data. 
***
Note that kCSD reconstruction is designed to work with arbitrary electrode setups. Removing specific electrodes does not change the situation significantly. We focus here on broken electrodes as we see it is a common enough situation in practice that it deserves a consideration. We want to show how one can gain intuition regarding ways and places in which reconstruction may go wrong, when we slightly disturb a setup we are familiar with.

### Remove broken electrodes

To test the effects of removed electrodes on reconstruction from a given setup we simulate this with a function `remove_electrodes` that takes all the electrode positions for this setup and the number of electrodes that are to be removed. In this example we remove the electrodes randomly. Like we did previously, to facilitate reproducibility we also pass a `broken_seed` variable, so that at each subsequent run the same electrodes are discarded. By changing this seed we select a different set of electrodes for removal.

In [ ]:
def remove_electrodes(ele_pos, num_broken, broken_seed=42):
    rstate = np.random.RandomState(broken_seed)
    rmv = rstate.choice(ele_pos.shape[0], num_broken,
                        replace=False)
    return np.delete(ele_pos, rmv, 0)

### Error in estimation with broken electrodes

After removing the broken electrodes we compute the estimation error to gauge the effect of electrode removal on reconstruction.

In [ ]:
ele_pos_5_removed = remove_electrodes(ele_pos, 5) # Discard 5 electrodes of 100

pots_5_removed = forward_method(ele_pos_5_removed,
                                csd_at,
                                true_csd)
k = do_kcsd(ele_pos_5_removed, pots_5_removed)
k.cross_validate(Rs=np.linspace(0.15, 0.2, 6),
                 lambdas=np.logspace(-7, -4, 7))

In [ ]:
err = point_errors(true_csd, k.values('CSD'))

ax = make_error_plot(k.estm_x, k.estm_y, err, 
                     title='Error CSD, 5 broken electrodes')

ax.scatter(ele_pos_5_removed[:, 0], ele_pos_5_removed[:, 1], 10, c='k')

## Exercises

### Exercise 1: visualise errors for other ground truth CSD

Try to use different seeds, replace `CSD.gauss_2d_large` instead of `CSD.gauss_2d_small`, etc.

### Exercise 1a: define your own CSD ground truth profile

Define the profile as a numpy array compatible with `csd_at` coordinates.

### Exercise 2: average error for multiple sources

The estimation error depends on the true CSD. To better understand the effects of the setup compute the average error across multiple sources - for several(dozens) different seeds. As an example here we show this for two seeds. This step is computationally expensive, however, it would normally be carried out only once for a given electrode design configuration. We believe this approach offers useful diagnostics and builds intuition regarding the estimation power for the given setup.

### Exercise 3: use nonregular distribution of electrodes.

Generate randomly 100 electrode position within the region of interest and compute potential there. Now, do reconstruction for increasing number of electrodes.
What is the result for a single electrode?
Do you see improvement of reconstruction? 
How can you decide if the reconstruction is good (enough)?

### Exercise 4: study the effect of noise on the reconstruction.

If you increase noise level, the reconstruction gets worse. Can you improve it by adding more electrodes? playing with regularization? for a given level of noise, is there a cap on the reconstruction error for a given ground truth?

### Exercise 4a: use different norms to quantify reconstruction errors

### Exercise 5: misplace the electrodes

Place the ground truth source in one place,  place electrodes elsewhere. Can you reconstruct the source from misplaced electrodes?

### Exercise 6: try other method of reconstruction parameters selection

Try to use `k.L_curve()` instead of `k.cross_validate()`.